In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import sys
import matplotlib.animation as animation
import glob, re, os
import pandas as pd

import tensorflow as tf
import seaborn as sns

from tensorflow import keras
from tensorflow.keras import layers
# Don't use cuda with keras
keras.backend.clear_session()

os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


import scienceplots
plt.style.use('default')
plt.style.use(['science', 'high-vis', 'grid'])

In [ ]:
def plot_loss(history, ax):
    """ Plot the loss function. """
    ax.plot(history.history['loss'], label='loss')
    ax.plot(history.history['val_loss'], label='val_loss')
    ax.grid(True)

In [ ]:
dataset_path_dir = "../../build/DATASET_COMPLEX_SPAD/"
list_of_files = glob.glob(dataset_path_dir + "*.csv")
latest_file = max(list_of_files, key=os.path.getctime)
dataset_path = latest_file
print("Dataset path: ", dataset_path)
df = pd.read_csv(dataset_path, sep=",", header=0)

idx_BV = df.columns.get_loc("BreakdownVoltage")
idxBrP = idx_BV + 1
idxDW = idx_BV + 2
idxDoping = np.arange(1, idx_BV, 1)
print(idxDoping)

In [ ]:
# Drop NaN values
df = df.dropna()
df.tail()

# The columns in idxDoping must be log10 transformed and the sign kept
df.iloc[:, idxDoping] =np.log10(df.iloc[:, idxDoping])

df.tail()

## Neural network regression to learn BV from Total Length, Donor Length, Donor Level and Acceptor level.

In [ ]:
# Neural network regression to learn BV from Total Length, Donor Length, Donor Level and Acceptor level.
# We will use the keras/TensorFlow library to build the neural network.

In [ ]:
# Split the data into train and test sets
train_dataset = df.sample(frac=0.5, random_state=0)
test_dataset = df.drop(train_dataset.index)


In [ ]:
# Inspect the data
pp = sns.pairplot(train_dataset[["TotalLength", "BreakdownVoltage", "BreakdownProbability", "DepletionWidth"]], diag_kind="kde")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1)
axs.scatter(train_dataset["BreakdownVoltage"], train_dataset["DepletionWidth"], s=2, marker='.', alpha=0.65, edgecolors='k', linewidths=0.1)
axs.set_xlabel("Breakdown Voltage (V)")
axs.set_ylabel("Depletion Width ($\mu$m)")
fig.savefig("BreakdownVoltage_vs_DepletionWidth_MC_Sampling.png", dpi=300)
fig.savefig("BreakdownVoltage_vs_DepletionWidth_MC_Sampling.pdf", dpi=300)
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 1)
axs.scatter(train_dataset["BreakdownVoltage"], train_dataset["BreakdownProbability"], s=2, marker='.', alpha=0.65, edgecolors='k', linewidths=0.1)
axs.set_xlabel("Breakdown Voltage (V)")
axs.set_ylabel("Breakdown Probability")
axs.set_ylim(0, 1)
fig.savefig("BreakdownVoltage_vs_BreakdownProbability_MC_Sampling.png", dpi=300)
fig.savefig("BreakdownVoltage_vs_BreakdownProbability_MC_Sampling.pdf", dpi=300)

plt.show()

In [ ]:
train_dataset.describe().transpose()

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

Labels_Features = ["BreakdownVoltage", "BreakdownProbability", "DepletionWidth"]

train_labels = train_features.pop(Labels_Features[0])
test_labels = test_features.pop(Labels_Features[0])

train_labels = train_features.pop(Labels_Features[1])
test_labels = test_features.pop(Labels_Features[1])

train_labels = train_features.pop(Labels_Features[2])
test_labels = test_features.pop(Labels_Features[2])

In [ ]:
train_dataset.describe().transpose()[['mean', 'std']]

In [ ]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(train_features))
print(normalizer.mean.numpy())


In [ ]:
# First try with a simple linear model
linear_model = tf.keras.Sequential([
    normalizer,
    layers.Dense(units=1)
])

linear_model.summary()

linear_model.predict(train_features[:10])
linear_model.layers[1].kernel

linear_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.1),
    loss='mean_absolute_error')

history = linear_model.fit(
    train_features,
    train_labels_BV,
    epochs=200,
    # suppress logging
    verbose=1,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

hist = pd.DataFrame(history.history)
hist['epoch'] = history.epoch
hist.tail()

fig, ax = plt.subplots()
plot_loss(history, ax)
ax.set_ylim(bottom=0.0)

plt.show()

In [ ]:
# Regression with a deep neural network (Input size is 201)
def build_and_compile_model(norm):
    model = keras.Sequential([
        norm,
        layers.Dense(2048, activation='relu'),
        layers.Dense(1024, activation='relu'),
        layers.Dense(512, activation='relu'),
        layers.Dense(256, activation='relu'),
        layers.Dense(128, activation='relu'),
        layers.Dense(64, activation='relu'),
        layers.Dense(32, activation='relu'), 
    ])

    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(0.001))
    return model



dnn_model_BV = build_and_compile_model(normalizer)
dnn_model_BV.summary()

history_BV = dnn_model_BV.fit(
    train_features, train_labels_BV,
    validation_split=0.2,
    verbose=1, epochs=250)


In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
plot_loss(history_BV, ax)
ax.set_ylim(bottom=-2.0)
# Inset zoom
# axins = ax.inset_axes([0.4, 0.4, 0.5, 0.5])
# plot_loss(history_BV, axins)
# axins.set_xlim(history_BV.epoch[-1]-50, history_BV.epoch[-1])
# axins.set_ylim(0.0, 2.0)
# ax.indicate_inset_zoom(axins, edgecolor="k", alpha=0.95)
ax.set_xlabel('Epoch')
ax.set_ylabel('Error [BV]')

fig.legend(loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.07), frameon=True, fancybox=True, shadow=False)
fig.tight_layout()
fig.savefig('BV_loss.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Make predictions
test_predictions_BV = dnn_model.predict(test_features).flatten()

fig, ax = plt.subplots(figsize=(4, 3))
ax.scatter(test_labels_BV, test_predictions_BV, s=1)
ax.set_xlabel('True Values [BV]')
ax.set_ylabel('Predictions [BV]')
ax.set_aspect('equal')
minmax_x = np.array([0, 1.1*np.max(test_labels_BV)])
ax.plot(minmax_x, minmax_x, color='k', linestyle='--', label='y=x')
ax.legend(loc='lower right')
plt.show()

In [ ]:
history_BrP = dnn_model.fit(
    train_features, train_labels_BrP,
    validation_split=0.2,
    verbose=1, epochs=250)

In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
plot_loss(history_BrP, ax)
ax.set_ylim(bottom=-0.10, top=2.0)
# Inset zoom
axins = ax.inset_axes([0.4, 0.4, 0.5, 0.5])
plot_loss(history_BrP, axins)
axins.set_xlim(history_BrP.epoch[-1]-50, history_BrP.epoch[-1])
axins.set_ylim(0.0, 0.10)
ax.indicate_inset_zoom(axins, edgecolor="k", alpha=0.95)
ax.set_xlabel('Epoch')
ax.set_ylabel('Error [BrP]')

fig.legend(loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.07), frameon=True, fancybox=True, shadow=False)
fig.tight_layout()
fig.savefig('BV_loss.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Make predictions
test_predictions_BrP = dnn_model.predict(test_features).flatten()

fig, ax = plt.subplots(figsize=(4, 3))
ax.scatter(test_labels_BrP, test_predictions_BrP, s=1)
ax.set_xlabel('True Values [BrP]')
ax.set_ylabel('Predictions [BrP]')
ax.set_aspect('equal')
minmax_x = np.array([0, 1.1*np.max(test_labels_BrP)])
ax.plot(minmax_x, minmax_x, color='k', linestyle='--', label='y=x')
ax.legend(loc='lower right')
plt.show()

In [ ]:
history_DW = dnn_model.fit(
    train_features, train_labels_DW,
    validation_split=0.2,
    verbose=1, epochs=250)


In [ ]:
fig, ax = plt.subplots(figsize=(4, 3))
plot_loss(history_DW, ax)
ax.set_ylim(bottom=-0.10, top=1.0)
# Inset zoom
axins = ax.inset_axes([0.4, 0.4, 0.5, 0.5])
plot_loss(history_DW, axins)
axins.set_xlim(history_DW.epoch[-1]-50, history_DW.epoch[-1])
axins.set_ylim(0.0, 0.1)
ax.indicate_inset_zoom(axins, edgecolor="k", alpha=0.95)
ax.set_xlabel('Epoch')
ax.set_ylabel('Error [DW]')

fig.legend(loc='lower center', ncol=2, bbox_to_anchor=(0.5, -0.07), frameon=True, fancybox=True, shadow=False)
fig.tight_layout()
fig.savefig('BV_loss.pdf', bbox_inches='tight')
plt.show()

In [ ]:
# Make predictions
test_predictions_DW = dnn_model.predict(test_features).flatten()

fig, ax = plt.subplots(figsize=(4, 3))
ax.scatter(test_labels_DW, test_predictions_DW, s=1)
ax.set_xlabel('True Values [DW]')
ax.set_ylabel('Predictions [DW]')
ax.set_aspect('equal')
minmax_x = np.array([0, 1.1*np.max(test_labels_DW)])
ax.plot(minmax_x, minmax_x, color='k', linestyle='--', label='y=x')
ax.legend(loc='lower right')
plt.show()
